# Comprehensive Exploratory Data Analysis (EDA) - Customer Churn Dataset

This notebook provides an in-depth exploratory data analysis of the customer churn dataset, covering:
- Data overview and structure
- Missing value analysis
- Univariate analysis (distributions)
- Bivariate analysis (relationships with churn)
- Multivariate analysis
- Outlier detection
- Correlation analysis
- Statistical insights

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from scipy.stats import chi2_contingency, normaltest, kstest, shapiro
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Set visualization styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 1. Data Loading and Initial Inspection

In [ ]:
# Load the dataset
df = pd.read_csv('customer_churn_dataset_with_date.csv')

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nNumber of samples: {df.shape[0]:,}")
print(f"Number of features: {df.shape[1]}")

In [ ]:
# Display first few rows
print("First 10 rows of the dataset:")
df.head(10)

In [ ]:
# Display last few rows
print("Last 10 rows of the dataset:")
df.tail(10)

In [ ]:
# Random sample
print("Random sample of 10 rows:")
df.sample(10, random_state=42)

In [ ]:
# Data types and non-null counts
print("Dataset Information:")
df.info()

In [ ]:
# Data types summary
print("\nData Types Summary:")
print(df.dtypes.value_counts())
print("\nDetailed breakdown:")
for dtype in df.dtypes.unique():
    cols = df.select_dtypes(include=[dtype]).columns.tolist()
    print(f"\n{dtype}: {len(cols)} columns")
    print(f"  {cols}")

In [ ]:
# Statistical summary for numerical features
print("Statistical Summary - Numerical Features:")
df.describe(include=[np.number]).T

In [ ]:
# Statistical summary for categorical features
print("Statistical Summary - Categorical Features:")
df.describe(include=['object']).T

In [ ]:
# Memory usage
print("Memory Usage:")
memory_usage = df.memory_usage(deep=True)
print(memory_usage)
print(f"\nTotal memory usage: {memory_usage.sum() / 1024**2:.2f} MB")

## 2. Missing Values Analysis

In [ ]:
# Missing values count and percentage
missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})

missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print("Missing Values Summary:")
print(missing_df)

In [ ]:
# Visualize missing values
if len(missing_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Bar plot of missing values
    missing_df.sort_values('Missing_Count', ascending=True).plot(
        kind='barh', x='Column', y='Missing_Count', ax=axes[0], color='coral', legend=False
    )
    axes[0].set_title('Missing Values Count by Feature', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Count of Missing Values')
    axes[0].set_ylabel('Features')
    
    # Percentage plot
    missing_df.sort_values('Missing_Percentage', ascending=True).plot(
        kind='barh', x='Column', y='Missing_Percentage', ax=axes[1], color='skyblue', legend=False
    )
    axes[1].set_title('Missing Values Percentage by Feature', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Percentage (%)')
    axes[1].set_ylabel('Features')
    
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found in the dataset!")

In [ ]:
# Missing value heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Missing Values Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Missing value patterns
print("Missing Value Patterns:")
print(f"Total rows with at least one missing value: {df.isnull().any(axis=1).sum():,}")
print(f"Percentage: {(df.isnull().any(axis=1).sum() / len(df)) * 100:.2f}%")
print(f"\nRows with all values present: {(~df.isnull().any(axis=1)).sum():,}")
print(f"Percentage: {((~df.isnull().any(axis=1)).sum() / len(df)) * 100:.2f}%")

## 3. Target Variable Analysis

In [ ]:
# Churn distribution
churn_counts = df['churn'].value_counts()
churn_pct = df['churn'].value_counts(normalize=True) * 100

print("Churn Distribution:")
churn_summary = pd.DataFrame({
    'Count': churn_counts,
    'Percentage': churn_pct
})
print(churn_summary)

print(f"\nChurn Rate: {churn_pct[1]:.2f}%")
print(f"Retention Rate: {churn_pct[0]:.2f}%")
print(f"\nClass Imbalance Ratio: 1:{churn_counts[0]/churn_counts[1]:.2f}")

In [ ]:
# Visualize churn distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Count plot
sns.countplot(data=df, x='churn', ax=axes[0], palette='Set2')
axes[0].set_title('Churn Distribution - Count', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churn (0=No, 1=Yes)')
axes[0].set_ylabel('Count')
for container in axes[0].containers:
    axes[0].bar_label(container)

# Pie chart
colors = ['#90EE90', '#FF6B6B']
axes[1].pie(churn_counts, labels=['Retained (0)', 'Churned (1)'], autopct='%1.1f%%', 
            startangle=90, colors=colors, explode=(0, 0.1))
axes[1].set_title('Churn Distribution - Percentage', fontsize=14, fontweight='bold')

# Percentage bar plot
churn_pct.plot(kind='bar', ax=axes[2], color=['#90EE90', '#FF6B6B'])
axes[2].set_title('Churn Distribution - Percentage', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Churn (0=No, 1=Yes)')
axes[2].set_ylabel('Percentage (%)')
axes[2].set_xticklabels(['Retained (0)', 'Churned (1)'], rotation=0)
for container in axes[2].containers:
    axes[2].bar_label(container, fmt='%.1f%%')

plt.tight_layout()
plt.show()

## 4. Univariate Analysis - Numerical Features

In [ ]:
# Identify numerical columns (excluding customer_id and churn)
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['customer_id', 'churn']]

print(f"Numerical features: {numerical_cols}")
print(f"Total: {len(numerical_cols)} features")

In [ ]:
# Detailed statistics for each numerical feature
print("Detailed Statistics for Numerical Features:\n")
for col in numerical_cols:
    print(f"\n{'='*60}")
    print(f"Feature: {col.upper()}")
    print(f"{'='*60}")
    
    # Remove missing values for analysis
    data = df[col].dropna()
    
    print(f"Count: {len(data):,}")
    print(f"Missing: {df[col].isnull().sum():,}")
    print(f"\nCentral Tendency:")
    print(f"  Mean: {data.mean():.3f}")
    print(f"  Median: {data.median():.3f}")
    print(f"  Mode: {data.mode().values[0] if len(data.mode()) > 0 else 'N/A'}")
    
    print(f"\nDispersion:")
    print(f"  Std Dev: {data.std():.3f}")
    print(f"  Variance: {data.var():.3f}")
    print(f"  Range: {data.max() - data.min():.3f}")
    print(f"  IQR: {data.quantile(0.75) - data.quantile(0.25):.3f}")
    print(f"  Coefficient of Variation: {(data.std() / data.mean() * 100):.2f}%")
    
    print(f"\nShape:")
    print(f"  Skewness: {data.skew():.3f}")
    print(f"  Kurtosis: {data.kurtosis():.3f}")
    
    print(f"\nQuantiles:")
    print(f"  Min: {data.min():.3f}")
    print(f"  Q1 (25%): {data.quantile(0.25):.3f}")
    print(f"  Q2 (50%): {data.quantile(0.50):.3f}")
    print(f"  Q3 (75%): {data.quantile(0.75):.3f}")
    print(f"  Max: {data.max():.3f}")

In [ ]:
# Distribution plots for numerical features
fig, axes = plt.subplots(len(numerical_cols), 3, figsize=(18, 5*len(numerical_cols)))

if len(numerical_cols) == 1:
    axes = axes.reshape(1, -1)

for idx, col in enumerate(numerical_cols):
    data = df[col].dropna()
    
    # Histogram with KDE
    axes[idx, 0].hist(data, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
    axes[idx, 0].set_title(f'{col} - Histogram', fontweight='bold')
    axes[idx, 0].set_xlabel(col)
    axes[idx, 0].set_ylabel('Frequency')
    axes[idx, 0].axvline(data.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    axes[idx, 0].axvline(data.median(), color='green', linestyle='--', linewidth=2, label='Median')
    axes[idx, 0].legend()
    
    # Box plot
    axes[idx, 1].boxplot(data, vert=True, patch_artist=True,
                         boxprops=dict(facecolor='lightblue', alpha=0.7),
                         medianprops=dict(color='red', linewidth=2))
    axes[idx, 1].set_title(f'{col} - Box Plot', fontweight='bold')
    axes[idx, 1].set_ylabel(col)
    axes[idx, 1].grid(axis='y', alpha=0.3)
    
    # Q-Q plot
    stats.probplot(data, dist="norm", plot=axes[idx, 2])
    axes[idx, 2].set_title(f'{col} - Q-Q Plot', fontweight='bold')
    axes[idx, 2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Violin plots for numerical features
fig, axes = plt.subplots(1, len(numerical_cols), figsize=(6*len(numerical_cols), 6))

if len(numerical_cols) == 1:
    axes = [axes]

for idx, col in enumerate(numerical_cols):
    sns.violinplot(y=df[col], ax=axes[idx], color='lightcoral')
    axes[idx].set_title(f'{col} - Violin Plot', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(col)
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Univariate Analysis - Categorical Features

In [ ]:
# Identify categorical columns (excluding Date and customer_id)
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ['Date', 'customer_id']]

print(f"Categorical features: {categorical_cols}")
print(f"Total: {len(categorical_cols)} features")

In [ ]:
# Detailed analysis of categorical features
print("Detailed Analysis of Categorical Features:\n")
for col in categorical_cols:
    print(f"\n{'='*60}")
    print(f"Feature: {col.upper()}")
    print(f"{'='*60}")
    
    value_counts = df[col].value_counts()
    value_pct = df[col].value_counts(normalize=True) * 100
    
    summary = pd.DataFrame({
        'Count': value_counts,
        'Percentage': value_pct
    })
    
    print(f"\nUnique values: {df[col].nunique()}")
    print(f"Missing values: {df[col].isnull().sum()}")
    print(f"\nValue Distribution:")
    print(summary)
    print(f"\nMode: {df[col].mode().values[0] if len(df[col].mode()) > 0 else 'N/A'}")

In [ ]:
# Visualize categorical features
fig, axes = plt.subplots(len(categorical_cols), 2, figsize=(16, 5*len(categorical_cols)))

if len(categorical_cols) == 1:
    axes = axes.reshape(1, -1)

for idx, col in enumerate(categorical_cols):
    # Count plot
    value_counts = df[col].value_counts()
    sns.countplot(data=df, x=col, ax=axes[idx, 0], palette='Set3', order=value_counts.index)
    axes[idx, 0].set_title(f'{col} - Distribution', fontsize=14, fontweight='bold')
    axes[idx, 0].set_xlabel(col)
    axes[idx, 0].set_ylabel('Count')
    axes[idx, 0].tick_params(axis='x', rotation=45)
    for container in axes[idx, 0].containers:
        axes[idx, 0].bar_label(container)
    
    # Pie chart
    axes[idx, 1].pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90)
    axes[idx, 1].set_title(f'{col} - Percentage Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Bivariate Analysis - Numerical Features vs Churn

In [ ]:
# Statistical comparison between churned and retained customers
print("Numerical Features Comparison: Churned vs Retained Customers\n")
print("="*80)

for col in numerical_cols:
    print(f"\nFeature: {col.upper()}")
    print("-"*80)
    
    churned = df[df['churn'] == 1][col].dropna()
    retained = df[df['churn'] == 0][col].dropna()
    
    print(f"\nRetained Customers (n={len(retained)}):")
    print(f"  Mean: {retained.mean():.3f}")
    print(f"  Median: {retained.median():.3f}")
    print(f"  Std: {retained.std():.3f}")
    
    print(f"\nChurned Customers (n={len(churned)}):")
    print(f"  Mean: {churned.mean():.3f}")
    print(f"  Median: {churned.median():.3f}")
    print(f"  Std: {churned.std():.3f}")
    
    print(f"\nDifference:")
    print(f"  Mean Difference: {churned.mean() - retained.mean():.3f}")
    print(f"  Percentage Change: {((churned.mean() - retained.mean()) / retained.mean() * 100):.2f}%")
    
    # Mann-Whitney U test (non-parametric)
    statistic, p_value = stats.mannwhitneyu(retained, churned, alternative='two-sided')
    print(f"\nMann-Whitney U Test:")
    print(f"  Test Statistic: {statistic:.3f}")
    print(f"  P-value: {p_value:.6f}")
    print(f"  Significant at α=0.05: {'Yes' if p_value < 0.05 else 'No'}")

In [ ]:
# Box plots: Numerical features by churn status
fig, axes = plt.subplots(2, len(numerical_cols), figsize=(6*len(numerical_cols), 12))

if len(numerical_cols) == 1:
    axes = axes.reshape(-1, 1)

for idx, col in enumerate(numerical_cols):
    # Box plot
    sns.boxplot(data=df, x='churn', y=col, ax=axes[0, idx], palette='Set2')
    axes[0, idx].set_title(f'{col} by Churn Status - Box Plot', fontweight='bold')
    axes[0, idx].set_xlabel('Churn (0=No, 1=Yes)')
    axes[0, idx].set_ylabel(col)
    axes[0, idx].grid(axis='y', alpha=0.3)
    
    # Violin plot
    sns.violinplot(data=df, x='churn', y=col, ax=axes[1, idx], palette='Set1')
    axes[1, idx].set_title(f'{col} by Churn Status - Violin Plot', fontweight='bold')
    axes[1, idx].set_xlabel('Churn (0=No, 1=Yes)')
    axes[1, idx].set_ylabel(col)
    axes[1, idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Distribution comparison: Churned vs Retained
fig, axes = plt.subplots(len(numerical_cols), 1, figsize=(14, 5*len(numerical_cols)))

if len(numerical_cols) == 1:
    axes = [axes]

for idx, col in enumerate(numerical_cols):
    churned = df[df['churn'] == 1][col].dropna()
    retained = df[df['churn'] == 0][col].dropna()
    
    axes[idx].hist(retained, bins=50, alpha=0.5, label='Retained (0)', color='green', edgecolor='black')
    axes[idx].hist(churned, bins=50, alpha=0.5, label='Churned (1)', color='red', edgecolor='black')
    axes[idx].set_title(f'{col} Distribution: Churned vs Retained', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Bivariate Analysis - Categorical Features vs Churn

In [ ]:
# Categorical features vs Churn - Statistical analysis
print("Categorical Features vs Churn - Chi-Square Test\n")
print("="*80)

for col in categorical_cols:
    print(f"\nFeature: {col.upper()}")
    print("-"*80)
    
    # Create contingency table
    contingency_table = pd.crosstab(df[col], df['churn'])
    print("\nContingency Table:")
    print(contingency_table)
    
    # Chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\nChi-Square Test Results:")
    print(f"  Chi-Square Statistic: {chi2:.3f}")
    print(f"  P-value: {p_value:.6f}")
    print(f"  Degrees of Freedom: {dof}")
    print(f"  Significant at α=0.05: {'Yes' if p_value < 0.05 else 'No'}")
    
    # Cramér's V (effect size)
    n = contingency_table.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
    print(f"  Cramér's V: {cramers_v:.3f}")
    
    # Churn rate by category
    churn_rate = df.groupby(col)['churn'].agg(['sum', 'count', 'mean'])
    churn_rate.columns = ['Churned_Count', 'Total_Count', 'Churn_Rate']
    churn_rate['Churn_Rate'] = churn_rate['Churn_Rate'] * 100
    print("\nChurn Rate by Category:")
    print(churn_rate)

In [ ]:
# Visualize categorical features vs churn
fig, axes = plt.subplots(len(categorical_cols), 2, figsize=(16, 5*len(categorical_cols)))

if len(categorical_cols) == 1:
    axes = axes.reshape(1, -1)

for idx, col in enumerate(categorical_cols):
    # Stacked bar chart
    churn_cross = pd.crosstab(df[col], df['churn'], normalize='index') * 100
    churn_cross.plot(kind='bar', stacked=True, ax=axes[idx, 0], color=['#90EE90', '#FF6B6B'])
    axes[idx, 0].set_title(f'{col} vs Churn - Stacked Percentage', fontsize=14, fontweight='bold')
    axes[idx, 0].set_xlabel(col)
    axes[idx, 0].set_ylabel('Percentage (%)')
    axes[idx, 0].legend(['Retained (0)', 'Churned (1)'], loc='best')
    axes[idx, 0].tick_params(axis='x', rotation=45)
    axes[idx, 0].grid(axis='y', alpha=0.3)
    
    # Grouped bar chart
    churn_cross_counts = pd.crosstab(df[col], df['churn'])
    churn_cross_counts.plot(kind='bar', ax=axes[idx, 1], color=['#90EE90', '#FF6B6B'])
    axes[idx, 1].set_title(f'{col} vs Churn - Count', fontsize=14, fontweight='bold')
    axes[idx, 1].set_xlabel(col)
    axes[idx, 1].set_ylabel('Count')
    axes[idx, 1].legend(['Retained (0)', 'Churned (1)'], loc='best')
    axes[idx, 1].tick_params(axis='x', rotation=45)
    axes[idx, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Churn rate by categorical feature
fig, axes = plt.subplots(1, len(categorical_cols), figsize=(7*len(categorical_cols), 6))

if len(categorical_cols) == 1:
    axes = [axes]

for idx, col in enumerate(categorical_cols):
    churn_rate = df.groupby(col)['churn'].mean() * 100
    churn_rate.sort_values(ascending=False).plot(kind='bar', ax=axes[idx], color='coral')
    axes[idx].set_title(f'Churn Rate by {col}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Churn Rate (%)')
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].axhline(df['churn'].mean()*100, color='red', linestyle='--', 
                      linewidth=2, label=f'Overall: {df["churn"].mean()*100:.1f}%')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)
    for container in axes[idx].containers:
        axes[idx].bar_label(container, fmt='%.1f%%')

plt.tight_layout()
plt.show()

## 8. Correlation Analysis

In [ ]:
# Correlation matrix for numerical features
corr_matrix = df[numerical_cols + ['churn']].corr()

print("Correlation Matrix:")
print(corr_matrix)

# Correlation with target variable
print("\nCorrelation with Churn (sorted by absolute value):")
churn_corr = corr_matrix['churn'].drop('churn').sort_values(key=abs, ascending=False)
print(churn_corr)

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation with churn - bar plot
plt.figure(figsize=(12, 6))
churn_corr_abs = churn_corr.abs().sort_values(ascending=True)
colors = ['green' if x > 0 else 'red' for x in churn_corr[churn_corr_abs.index]]
churn_corr[churn_corr_abs.index].plot(kind='barh', color=colors, edgecolor='black')
plt.title('Correlation of Features with Churn', fontsize=16, fontweight='bold')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.axvline(0, color='black', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Multivariate Analysis

In [ ]:
# Pair plot for numerical features (sample for performance)
sample_df = df.sample(n=min(1000, len(df)), random_state=42)
sns.pairplot(sample_df[numerical_cols + ['churn']], hue='churn', palette='Set1', 
             diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Pair Plot - Numerical Features (Sample)', y=1.02, fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots matrix
if len(numerical_cols) >= 2:
    from itertools import combinations
    
    # Get all pairs of numerical features
    feature_pairs = list(combinations(numerical_cols, 2))
    
    # Limit to first 6 pairs for visualization
    n_pairs = min(6, len(feature_pairs))
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for idx, (feat1, feat2) in enumerate(feature_pairs[:n_pairs]):
        for churn_val in [0, 1]:
            subset = df[df['churn'] == churn_val]
            axes[idx].scatter(subset[feat1], subset[feat2], 
                            label=f'Churn={churn_val}', alpha=0.5, s=20)
        
        axes[idx].set_xlabel(feat1, fontsize=10)
        axes[idx].set_ylabel(feat2, fontsize=10)
        axes[idx].set_title(f'{feat1} vs {feat2}', fontweight='bold')
        axes[idx].legend()
        axes[idx].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 3D scatter plot (if we have at least 3 numerical features)
if len(numerical_cols) >= 3:
    from mpl_toolkits.mplot3d import Axes3D
    
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    sample_df = df.sample(n=min(5000, len(df)), random_state=42)
    
    colors = ['green' if x == 0 else 'red' for x in sample_df['churn']]
    ax.scatter(sample_df[numerical_cols[0]], 
               sample_df[numerical_cols[1]], 
               sample_df[numerical_cols[2]], 
               c=colors, alpha=0.5, s=20)
    
    ax.set_xlabel(numerical_cols[0], fontsize=10)
    ax.set_ylabel(numerical_cols[1], fontsize=10)
    ax.set_zlabel(numerical_cols[2], fontsize=10)
    ax.set_title(f'3D Scatter: {numerical_cols[0]} vs {numerical_cols[1]} vs {numerical_cols[2]}',
                 fontsize=14, fontweight='bold')
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [Line2D([0], [0], marker='o', color='w', 
                              markerfacecolor='green', markersize=10, label='Retained'),
                      Line2D([0], [0], marker='o', color='w', 
                              markerfacecolor='red', markersize=10, label='Churned')]
    ax.legend(handles=legend_elements)
    
    plt.tight_layout()
    plt.show()

## 10. Outlier Detection and Analysis

In [ ]:
# Outlier detection using IQR method
print("Outlier Detection using IQR Method\n")
print("="*80)

outlier_summary = []

for col in numerical_cols:
    data = df[col].dropna()
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    
    print(f"\nFeature: {col.upper()}")
    print(f"  Q1: {Q1:.3f}")
    print(f"  Q3: {Q3:.3f}")
    print(f"  IQR: {IQR:.3f}")
    print(f"  Lower Bound: {lower_bound:.3f}")
    print(f"  Upper Bound: {upper_bound:.3f}")
    print(f"  Number of Outliers: {len(outliers):,}")
    print(f"  Percentage of Outliers: {(len(outliers) / len(data) * 100):.2f}%")
    
    outlier_summary.append({
        'Feature': col,
        'Outlier_Count': len(outliers),
        'Outlier_Percentage': (len(outliers) / len(data) * 100),
        'Lower_Bound': lower_bound,
        'Upper_Bound': upper_bound
    })

outlier_df = pd.DataFrame(outlier_summary)
print("\n" + "="*80)
print("\nOutlier Summary:")
print(outlier_df)

In [ ]:
# Visualize outliers
fig, axes = plt.subplots(1, len(numerical_cols), figsize=(6*len(numerical_cols), 6))

if len(numerical_cols) == 1:
    axes = [axes]

for idx, col in enumerate(numerical_cols):
    data = df[col].dropna()
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Box plot with outliers highlighted
    bp = axes[idx].boxplot(data, vert=True, patch_artist=True,
                           boxprops=dict(facecolor='lightblue'),
                           flierprops=dict(marker='o', markerfacecolor='red', 
                                         markersize=5, alpha=0.5))
    
    axes[idx].set_title(f'{col} - Outlier Detection', fontweight='bold')
    axes[idx].set_ylabel(col)
    axes[idx].axhline(lower_bound, color='orange', linestyle='--', 
                      linewidth=2, label='Lower Bound')
    axes[idx].axhline(upper_bound, color='orange', linestyle='--', 
                      linewidth=2, label='Upper Bound')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Z-score method for outlier detection
print("Outlier Detection using Z-Score Method (|z| > 3)\n")
print("="*80)

for col in numerical_cols:
    data = df[col].dropna()
    z_scores = np.abs(stats.zscore(data))
    outliers = data[z_scores > 3]
    
    print(f"\nFeature: {col.upper()}")
    print(f"  Number of Outliers (|z| > 3): {len(outliers):,}")
    print(f"  Percentage: {(len(outliers) / len(data) * 100):.2f}%")

## 11. Normality Tests

In [ ]:
# Normality tests for numerical features
print("Normality Tests for Numerical Features\n")
print("="*80)

normality_results = []

for col in numerical_cols:
    data = df[col].dropna()
    
    # Sample for large datasets (Shapiro-Wilk works best with n < 5000)
    if len(data) > 5000:
        sample_data = data.sample(5000, random_state=42)
    else:
        sample_data = data
    
    print(f"\nFeature: {col.upper()}")
    print("-"*80)
    
    # Shapiro-Wilk test
    stat_sw, p_sw = shapiro(sample_data)
    print(f"\nShapiro-Wilk Test:")
    print(f"  Test Statistic: {stat_sw:.6f}")
    print(f"  P-value: {p_sw:.6f}")
    print(f"  Normal at α=0.05: {'Yes' if p_sw > 0.05 else 'No'}")
    
    # Kolmogorov-Smirnov test
    stat_ks, p_ks = kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f"\nKolmogorov-Smirnov Test:")
    print(f"  Test Statistic: {stat_ks:.6f}")
    print(f"  P-value: {p_ks:.6f}")
    print(f"  Normal at α=0.05: {'Yes' if p_ks > 0.05 else 'No'}")
    
    # D'Agostino's K-squared test
    if len(data) >= 8:  # Minimum sample size for this test
        stat_da, p_da = normaltest(data)
        print(f"\nD'Agostino-Pearson Test:")
        print(f"  Test Statistic: {stat_da:.6f}")
        print(f"  P-value: {p_da:.6f}")
        print(f"  Normal at α=0.05: {'Yes' if p_da > 0.05 else 'No'}")
    
    normality_results.append({
        'Feature': col,
        'Shapiro_Wilk_p': p_sw,
        'KS_Test_p': p_ks,
        'Is_Normal': 'Yes' if (p_sw > 0.05 and p_ks > 0.05) else 'No'
    })

normality_df = pd.DataFrame(normality_results)
print("\n" + "="*80)
print("\nNormality Test Summary:")
print(normality_df)

## 12. Time Series Analysis (if Date column is meaningful)

In [ ]:
# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Churn over time
churn_over_time = df.groupby('Date')['churn'].agg(['sum', 'count', 'mean']).reset_index()
churn_over_time.columns = ['Date', 'Churned_Count', 'Total_Count', 'Churn_Rate']
churn_over_time['Churn_Rate'] = churn_over_time['Churn_Rate'] * 100

print("Churn Over Time - First 10 Days:")
print(churn_over_time.head(10))

In [ ]:
# Plot churn over time
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Churn count over time
axes[0].plot(churn_over_time['Date'], churn_over_time['Churned_Count'], 
             marker='o', linewidth=2, markersize=4, color='red')
axes[0].set_title('Number of Churned Customers Over Time', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Count')
axes[0].grid(alpha=0.3)

# Churn rate over time
axes[1].plot(churn_over_time['Date'], churn_over_time['Churn_Rate'], 
             marker='o', linewidth=2, markersize=4, color='blue')
axes[1].axhline(df['churn'].mean()*100, color='red', linestyle='--', 
                linewidth=2, label=f'Overall: {df["churn"].mean()*100:.2f}%')
axes[1].set_title('Churn Rate Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Key Insights and Summary

In [ ]:
print("="*80)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("="*80)

print("\n1. DATASET OVERVIEW:")
print(f"   - Total Records: {len(df):,}")
print(f"   - Total Features: {df.shape[1]}")
print(f"   - Numerical Features: {len(numerical_cols)}")
print(f"   - Categorical Features: {len(categorical_cols)}")

print("\n2. TARGET VARIABLE (CHURN):")
print(f"   - Churn Rate: {df['churn'].mean()*100:.2f}%")
print(f"   - Retention Rate: {(1-df['churn'].mean())*100:.2f}%")
print(f"   - Class Imbalance Ratio: 1:{df['churn'].value_counts()[0]/df['churn'].value_counts()[1]:.2f}")

print("\n3. MISSING VALUES:")
total_missing = df.isnull().sum().sum()
if total_missing > 0:
    print(f"   - Total Missing Values: {total_missing:,}")
    print(f"   - Features with Missing Values: {(df.isnull().sum() > 0).sum()}")
    print(f"   - Percentage of Data Missing: {(total_missing / (df.shape[0] * df.shape[1]) * 100):.2f}%")
else:
    print("   - No missing values detected")

print("\n4. CORRELATIONS WITH CHURN:")
print("   Top Features (by absolute correlation):")
churn_corr_sorted = corr_matrix['churn'].drop('churn').abs().sort_values(ascending=False)
for i, (feat, corr) in enumerate(churn_corr_sorted.head(5).items(), 1):
    print(f"   {i}. {feat}: {corr:.3f}")

print("\n5. OUTLIERS:")
print("   Features with significant outliers (>5%):")
for _, row in outlier_df[outlier_df['Outlier_Percentage'] > 5].iterrows():
    print(f"   - {row['Feature']}: {row['Outlier_Percentage']:.2f}%")

print("\n6. DISTRIBUTION CHARACTERISTICS:")
for col in numerical_cols:
    skew = df[col].skew()
    print(f"   - {col}: Skewness = {skew:.3f} ({'Right-skewed' if skew > 0.5 else 'Left-skewed' if skew < -0.5 else 'Approximately symmetric'})")

print("\n" + "="*80)
print("END OF EDA")
print("="*80)